# 1st part - Capstone Project

In [1]:
!conda install -c conda-forge geopy --yes

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  24.35 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  35.76 MB/s


In [2]:
!conda install -c conda-forge folium=0.5.0 --yes

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  55.50 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  36.18 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  40.03 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  49.43 MB/s


In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from bs4 import BeautifulSoup

In [4]:
postal_codes_html = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
postal_codes_data_raw = BeautifulSoup(postal_codes_html, 'lxml')
postal_codes_data = postal_codes_data_raw.find('table', class_='wikitable sortable')

table_list = []
for rows in postal_codes_data.find_all('td'):
    row = rows.text
    row = row.replace('\n', '')
    table_list.append(row)

In [5]:
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 
postal_codes = []
postal_codes = pd.DataFrame(columns=column_names)

In [6]:
postal_codes.iloc[:,0] = table_list[::3]
postal_codes.iloc[:,1] = table_list[1::3]
postal_codes.iloc[:,2] = table_list[2::3]
postal_codes.replace("Not assigned", np.nan, inplace = True)
postal_codes.dropna(subset=["Borough"], axis=0, inplace = True)
postal_codes.reset_index(drop=True, inplace=True)

In [7]:
for i in range(0, postal_codes.shape[0]):
    if pd.isnull(postal_codes.loc[i,'Neighborhood']):
        postal_codes.replace(postal_codes.loc[i,'Neighborhood'], postal_codes.loc[i,'Borough'],inplace=True)
postal_codes.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [8]:
postal_codes['Neighborhood'] = postal_codes[['PostalCode','Borough','Neighborhood']].groupby(['PostalCode','Borough'])['Neighborhood'].transform(lambda x: ','.join(x)) 
postal_codes.drop_duplicates(inplace=True)
postal_codes.reset_index(drop=True, inplace=True)

postal_codes.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [9]:
postal_codes.shape

(103, 3)